In [13]:
import docker
from IPython.display import display_html  # Jupyter
import logging
import sys
import functools
import inspect
import re

# Configure logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

def create_dockerfile(requirements_file="requirements.txt", script_name="app.py", workdir="/app", base_image="python:3.8-slim", function_args=""):
    """
    Create a Dockerfile with the specified configuration.

    Args:
        requirements_file (str): Path to the requirements file.
        script_name (str): Name of the script to run in the Docker container.
        workdir (str): Working directory inside the container.
        base_image (str): Base Docker image.

    Returns:
        None
    """
    dockerfile_content = f"""
        FROM {base_image}
        WORKDIR {workdir}
        COPY . {workdir}
        RUN pip install --no-cache-dir -r {requirements_file}
        EXPOSE 80
        CMD ["python3", "-c", "import {script_name}; {script_name}.{script_name}({function_args})"]
    """

    with open("Dockerfile", "w") as dockerfile:
        dockerfile.write(dockerfile_content)

def build_docker_image(image_name):
    """
    Build a Docker image with the specified name.

    Args:
        image_name (str): Name of the Docker image.

    Returns:
        tuple: A tuple containing the Docker image and build logs.
    """
    from IPython.display import display_html
    client = docker.from_env()

    logger.info(f"Building Docker image: {image_name}")
    return client.images.build(path=".", tag=image_name)

def run_docker_container(image, ports, detach=True):
    """
    Run a Docker container based on the given image.

    Args:
        image (str): ID or name of the Docker image.
        ports (dict): Port mapping for the container.
        detach (bool): Whether to run the container in the background.

    Returns:
        docker.models.containers.Container: The running Docker container.
    """
    client = docker.from_env()

    logger.info(f"Running Docker image: {image}")
    container = client.containers.run(image=image, ports=ports, detach=detach, stdin_open=True)

    return container

def stop_and_remove_container(container):
    """
    Stop and remove a Docker container.

    Args:
        container (docker.models.containers.Container): The Docker container to stop and remove.

    Returns:
        None
    """
    logger.info(f"Stopping and removing Docker container: {container.id}")
    container.stop()
    container.remove()

def create_requirements_txt():
    """
    Create a requirements.txt file if it doesn't exist.

    Returns:
        None
    """
    try:
        with open("requirements.txt", "r"):
            pass  # If the file exists, do nothing
    except FileNotFoundError:
        with open("requirements.txt", "w") as requirements_file:
            requirements_file.write("numpy==1.18.5\n")
            requirements_file.write("requests==2.24.0\n")
            # Add other dependencies as needed

def remove_dockerize_decorator(source_code):
    """
    Remove @dockerize decorator and anything on the same line from the beginning of the input string.

    Args:
        source_code (str): Input string containing source code.

    Returns:
        str: Source code with @dockerize decorator and anything on the same line removed.
    """
    # Define a regular expression pattern to match @dockerize and anything on the same line
    pattern = r'@dockerize[^\n]*\n?'

    # Use re.sub to replace the matched pattern with an empty string
    modified_source = re.sub(pattern, '', source_code)

    return modified_source

def get_function_source(func_name):
    """
    Get the source code of a function.

    Args:
        func_name (str): Name of the function.

    Returns:
        str: Source code of the function.
    """
    try:
        func = globals()[func_name]
        return remove_dockerize_decorator(inspect.getsource(func))
    except KeyError:
        return f"Function '{func_name}' not found"

def function_to_py(func_name):
    """
    Convert a function to a Python file.

    Args:
        func_name (str): Name of the function.

    Returns:
        None
    """
    content = get_function_source(func_name)
    file_path = f"{func_name}.py"
    try:
        with open(file_path, 'w') as file:
            file.write(content)
        print(f"Content successfully written to {file_path}")
    except Exception as e:
        print(f"Error writing to {file_path}: {str(e)}")

def dockerize(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Generate the Python script
        function_to_py(func.__name__)

        # Create other necessary files
        create_requirements_txt()
        create_dockerfile(script_name=func.__name__)

        # Build the Docker image
        docker_image_name = f"{func.__name__}_image"
        image, _ = build_docker_image(docker_image_name)

        # Run the Docker container
        container = run_docker_container(image.id, ports={'80/tcp': 4000}, detach=True)
        logger.info(f"Docker container ID: {container.id}")

        try:
            result = func(*args, **kwargs)
            return result
        finally:
            # Stop and remove the Docker container when the function execution is done
            stop_and_remove_container(container)

    return wrapper

@dockerize
def foo():
    print("foo")

if __name__ == "__main__":
    # Configure logging for the script
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logger = logging.getLogger(__name__)

    # Execute the dockerized function
    foo()

Content successfully written to foo.py
INFO:__main__:Building Docker image: foo_image
